In [1]:
import os
import json
import time
import planet
import requests

from requests.auth import HTTPBasicAuth

In [2]:
api_key = !planet auth value
os.environ['PL_API_KEY']=api_key[0]
API_KEY = os.getenv('PL_API_KEY')

In [7]:
def get_image_ids(geometry, start_date, end_date, item_type="PSScene"):
    
    """gets all image ids for a given time (start_date, end_date) and for a given geometry"""
    
    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geometry
    }

    # get images acquired within a date range
    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": f"{start_date}T00:00:00.000Z",
        "lt": f"{end_date}T00:00:00.000Z"
      }
    }

    
    # combine geo, date, cloud filters
    combined_filter = {
      "type": "AndFilter",
      "config": [geometry_filter, date_range_filter]
    }
    
    
    # more item_types reference: https://developers.planet.com/docs/apis/data/items-assets/#item-types

    # API request object
    search_request = {
      "item_types": [item_type], 
      "filter": combined_filter
    }

    search_result = \
      requests.post(
        'https://api.planet.com/data/v1/quick-search',
        auth=HTTPBasicAuth(API_KEY, ''),
        json=search_request)

    
    result = search_result.json()
    

    return [feature['id'] for feature in result['features']]

In [ ]:
def get_orders(roi_path, gee_project, gee_collection, start, end, extension=''):
    
    """Returns a list of all the orders requests"""

    orders = []
    
    # roi_path is the folder with each geojson geometry
    for roi_file in [os.path.join(roi_path, f) for f in os.listdir(roi_path) if f.endswith('.json')]:

    
        with open(roi_file, 'r') as f:
            geom = json.loads(f.read())
        
        # get scenes for each roi
        scene_ids = get_image_ids(geom, start, end, item_type='PSScene')

        
        cloud_config = planet.order_request.google_earth_engine(
            project=gee_project, collection=gee_collection
        )

        delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

        # harmonization and clip tools
        tools = [
            planet.order_request.clip_tool(geom),
            planet.order_request.harmonize_tool('Sentinel-2')
        ]
        
        
        data_products = [
            planet.order_request.product(item_ids=scene_ids,
                                     product_bundle='analytic_8b_sr_udm2',
                                     item_type='PSScene')
        ]
            
        order_name = os.path.basename(roi_file).split('.')[0] + extension
        
        order = planet.order_request.build_request(name=order_name,
                                                    products=data_products,
                                                    delivery=delivery_config,
                                                    tools=tools)
        
        orders.append(order)
        
    return orders


In [ ]:
def place_order(request, auth):
    
    """Places the order by sending a POST request to the Planet API"""
    
    headers = {'content-type': 'application/json'}
    orders_url = 'https://api.planet.com/compute/ops/orders/v2' 
    
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
#     print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

In [ ]:
# Modify the options below when using this script
##################################################
ROI_FOLDER = 'reclamation_sites'
GEE_PROJECT = 'planetscope-2' # ensure planet has write assess in Google Cloud Platform for the project
GEE_COLLECTION = 'all_rec_sites'
START_DATE = '2022-06-01'
END_DATE = '2022-10-20'

## optional params for naming
extension = '_'
##################################################

order_arr = get_orders(ROI_FOLDER, GEE_PROJECT, GEE_COLLECTION, START_DATE, END_DATE, extension=extension)

In [ ]:
######## Uncomment below to run script 

# for order in order_arr:
    
#     session = requests.Session()
#     session.auth = (API_KEY, "")
#     order_url = place_order(order, session.auth)
    
#     time.sleep(1)